In [9]:
%matplotlib inline
import csv
import numpy as np
import pickle
import sklearn
from sklearn import preprocessing,cross_validation,tree,metrics,decomposition,linear_model,pipeline,svm,grid_search,learning_curve
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from scipy.interpolate import spline
import xlrd
import math
import random
import pandas as pd
from numpy.random import choice
from scipy import stats
from collections import defaultdict
from operator import itemgetter, sub
from sklearn.metrics import roc_curve, auc, accuracy_score, explained_variance_score, \
    mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import matplotlib.mlab as mlab   

In [2]:
# Linear regression
def linear_regressor():
    linear_regression = linear_model.LinearRegression()
    return linear_regression

# LASSO regression
def lasso_regressor():
    lasso_params = {'alpha':[0.001,0.1,1.0,10.0,100.0]}
    grid_lasso = grid_search.GridSearchCV(linear_model.Lasso(),lasso_params)
    return grid_lasso
    
# SVM regressor
def svr():
    svm_params = {'C':[1.0,10.0,100.0],'epsilon':[0.01,0.1,1.0],'kernel':['linear','poly','rbf','sigmoid'],
                  'degree':[2,3,5],'gamma':[0.001,0.01,0.1],'coef0':[0.0,1.0]}
    grid_svm = grid_search.GridSearchCV(svm.SVR(),svm_params)
    return svm.SVR(C=1000)

# Decision tree regressor
def dtree():
    tree_params = {'max_depth':[50,100],'min_samples_leaf':[2,3,5,10]}
    grid_tree = grid_search.GridSearchCV(sklearn.tree.DecisionTreeRegressor(),tree_params)
    return grid_tree

# Baysian Ridge Regressor
def bayseian():
    bayseian_params = {'n_iter':[300,500],'alpha_1':[1e-5,1e-6],'alpha_2':[1e-5,1e-6],'lambda_1':[1e-5,1e-6],
                      'lambda_2':[1e-5,1e-6]}
    grid_bay = grid_search.GridSearchCV(linear_model.BayesianRidge(),bayseian_params)
    return grid_bay
    
# Logistic Regressor
def logistic():
    log_params = {'penalty':['l1','l2'],'C':[1.0,10.0,100.0]}
    grid_log = grid_search.GridSearchCV(linear_model.LogisticRegression(),log_params)
    return grid_log

# Plot the performance of regressors
# Input: key = 1 is the R^2 score, key = 2 is the MSE

def neuralnet():
    neural_params = {'hidden_layer_sizes':[10,20,50],'activation':['logistic'],'alpha':[0.0001],'max_iter':[2000,20000]}
    grid_neural = grid_search.GridSearchCV(MLPRegressor(),neural_params)
    return grid_neural




In [3]:
class FileToSamples:
    'convert the input .csv file to samples(Dataframe):\
    [Iteration,CellId,PilotPower,Load,Users,TBS,NB1Id*5,NB1Power*5,NB1Users*5,NB1Load*5\
    AveragePower,AverageLoad,AverageUser,TimeAveragePower,TimeAverageLoad,TimeAverageUser] '
    
    def __init__(self,filename):
        self.samples = pd.read_pickle(filename) # type DataFrame (pandas package)  
        # use for test feature selection
        self.feature = FileToSamples.__getFeatues(self.samples)  # type DataFrame (pandas package) 
        self.target=self.samples['TBS']
    @staticmethod
    def __getFeatues(samples):
        Testnumber =1
        feature=[]
        #nnEstimater = [TrainingNNModel()]*Testnumber
        feature=[pd.DataFrame()]*Testnumber
        feature[0]= samples[['PilotPower','TimeAverageLoad','TimeAverageUsers','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power']] 
        return feature


In [4]:
def get_score(regressor):
    
    trainingFile = 'Data__Random3.csv.pkl'
    #trainingFile = 'Workbook2.csv'
    testFile = 'Data__Random4.csv.pkl'
    
    testData = FileToSamples(testFile)
    trainData = FileToSamples(trainingFile)
    scaler = StandardScaler()  
    
    X_train= trainData.feature[0]
    X_test= testData.feature[0]
    y_train=trainData.target
    y_test=testData.target
        
    model = regressor.fit(X_train,y_train)
    score = model.score(X_test,y_test)
    y_pred = model.predict(X_test)
    mse = metrics.mean_squared_error(y_test,y_pred) # Mean squared error        
    return model, score, mse

In [5]:
result=6*[None]
score=6*[None]
mse=6*[None]
regressor=[linear_regressor(),lasso_regressor(),svr(),dtree(),bayseian(),neuralnet()]

for i in range(6):
    result[i],score[i],mse[i]=get_score(regressor[i])

In [16]:
score

[0.5102492736727265,
 0.5102565163897443,
 0.6079625569440468,
 0.4146100607945446,
 0.5102844485744826,
 0.8241619106480704]

In [17]:
mse

[8393.825569339597,
 8393.701436592002,
 6719.120027323015,
 10032.98367025468,
 8393.222707569543,
 3013.6846585904755]